In [1]:
def hint(string): return print(''.join(map(chr, map(lambda x: int(x, 16), string.split('.')))))

## 第一部分 背景与问题描述

## Environment
+ jupyter notebook (测试、试验环境)
+ Pycharm （开发环境）
+ python3.6
+ networkx
+ jieba
+ numpy, pandas, matplotlib
+ gensim
+ (optional) bottle

## Background

在自然语言处理中，有一个常见的问题就是对客户的评价进行分析。 这些用户评论中，包含了大量的有用信息，例如情感分析，或者相关事实描述。 例如，
> `“味道不错的面馆，性价比也相当之高，分量很足～女生吃小份，胃口小的，可能吃不完呢。环境在面馆来说算是好的，至少看上去堂子很亮，也比较干净，一般苍蝇馆子还是比不上这个卫生状况的。中午饭点的时候，人很多，人行道上也是要坐满的，隔壁的冒菜馆子，据说是一家，有时候也会开放出来坐吃面的人。“`

首先情感是正向的，除此之外我们还能够进行知道这个的几个事实描述：1. 性价比比较高； 2. 装修比较好； 3. 分量足。 

这些信息是非常重要宝贵的，不论是对于公司进行商业分析或者要建立一个搜索引擎排序，这些信息都是重要的参考因素。 那么在这个时候，我们就需要进行文本的情感分类了。

**注**: 此次问题的来源数据集来源于大众点评， 这个数据集也被用作 AI 全球挑战赛的数据集。 而且细粒度情感分类这个问题其实在目前而言是一个 *未解之谜*， 人类现在对这个问题并没有什么特别好的方法， 因为人的情感表达真的是很有变化的，例如“我不认为这个地方不好是不对的”。所以这个问题也需要大家在之后做出来基础模型之后，大家多想想办法，*八仙过海，各显神通*

所以，我们这一次是要使用深度学习的方法，建立一个模型，这个模型能够将一句话进行分类判断，判断出来这句话到底表达了什么重要信息。 说实话，这个看似简单的问题，曾经是困扰了科学家数十年的问题，就算是现在，深度学习，人工智能有了很大的进步，其效果也达不到人们预期的那么好,但是比起前些年已经好多了:)

这个问题我们希望的是，输入一句话，输出是这句话对于以下6大类，20小类进行打标，对于每个小类而言，都会有<** 正面情感, 中性情感, 负面情感, 情感倾向未提及 > ** 这4个类别。 

总得来说，我们现在这6大类，20小类的类别如下：

+ 位置: location
    + 交通是否便利(traffic convenience)
    + 距离商圈远近(distance from business district)
    + 是否容易寻找(easy to find)
+ 服务(service)	
    + 排队等候时间(wait time)
    + 服务人员态度(waiter’s attitude)
    + 是否容易停车(parking convenience)
    + 点菜/上菜速度(serving speed)
+ 价格(price)	
    + 价格水平(price level)
    + 性价比(cost-effective)
    + 折扣力度(discount)
+ 环境(environment)	
    + 装修情况(decoration)
    + 嘈杂情况(noise)
    + 就餐空间(space)
    + 卫生情况(cleaness)
+ 菜品(dish)	
    + 分量(portion)
    + 口感(taste)
    + 外观(look)
    + 推荐程度(recommendation)
+ 其他(others)	
    + 本次消费感受(overall experience)
    + 再次消费的意愿(willing to consume again)

而为了方便训练数据的标标注，训练数据中，<** 正面情感, 中性情感, 负面情感, 情感倾向未提及 > ** 分别对应与 (1, 0, -1, -2). 

例如说，
> `“味道不错的面馆，性价比也相当之高，分量很足～女生吃小份，胃口小的，可能吃不完呢。环境在面馆来说算是好的，至少看上去堂子很亮，也比较干净，一般苍蝇馆子还是比不上这个卫生状况的。中午饭点的时候，人很多，人行道上也是要坐满的，隔壁的冒菜馆子，据说是一家，有时候也会开放出来坐吃面的人。“`

这句话在训练数据中的标签就是：

+ 交通是否便利(traffic convenience)	-2 
+ 距离商圈远近(distance from business district)	-2
+ 是否容易寻找(easy to find)	-2
+ 排队等候时间(wait time)	-2
+ 服务人员态度(waiter’s attitude)	-2
+ 是否容易停车(parking convenience)	-2
+ 点菜/上菜速度(serving speed)	-2
+ 价格水平(price level)	-2
+ 性价比(cost-effective)	1
+ 折扣力度(discount)	-2
+ 装修情况(decoration)	1
+ 嘈杂情况(noise)	-2
+ 就餐空间(space)	-2
+ 卫生情况(cleaness)	1
+ 分量(portion)	1
+ 口感(taste)	1
+ 外观(look)	-2
+ 推荐程度(recommendation)	-2
+ 次消费感受(overall experience)	1
+ 再次消费的意愿(willing to consume again)	-2

## 数据集下载

数据集的下载在 https://challenger.ai/competition/fsauor2018， 大家下载数据集， 以及测试集，注意，训练集我们需要分成 training data, validation data 然后 test data里边的数据绝对不能在训练的时候用。 否则的话就是去了意义。 

**注**, 这个数据集之所以被用到了 AI 挑战赛中，是因为其难度很大。 绝大数人在公司中遇到的问题难度**不会**超过这个问题。 

## 评价

参照 https://challenger.ai/competition/fsauor2018, 这个问题的评价其实就是多个分类的f1 score 的平均值。

## 第二部分 基础理论部分

#### Q1: 机器学习中的Loss函数的作用为何？

回答：为了衡量当前模型的性能；具体衡量方式是计算当前模型的预测值与正确答案之间的距离。

In [2]:
# remove the # before hint, what you find? 
hint('8fd9.4e2a.7b54.6848.53ef.4ee5.5199.5f88.591a.ff0c.4f46.662f.4e3b.8981.662f.8981.6d89.53ca.5230.ff1a.4c.6f.73.73.20.51fd.6570.7528.6765.8861.91cf.673a.5668.5b66.4e60.8fc7.7a0b.4e2d.6a21.578b.8868.73b0.7684.ff0c.6211.4eec.901a.8fc7.20.4c.6f.73.73.20.51fd.6570.6765.8fdb.884c.4f18.5316.6a21.578b.6216.8005.9009.62e9.6a21.578b')

这个答案可以写很多，但是主要是要涉及到：Loss 函数用来衡量机器学习过程中模型表现的，我们通过 Loss 函数来进行优化模型或者选择模型


####  Q2: 为什么 SVM 适合核函数的方法？（考虑基于拉格朗日距离的 SVM 的 Loss 函数）

回答：

In [3]:
#hint('8be6.60c5.8bf7.53c2.8003.6211.4eec.7684.8bfe.7a0b.89c6.9891.ff0c.4e00.4e2a.4e3b.8981.7684.70b9.662f.ff0c.6700.540e.8bc1.660e.20.53.56.4d.20.6a21.578b.7684.6027.80fd.53ea.4e0e.20.78.5f.69.78.5f.6a.20.7684.4e58.673a.76f8.5173.ff0c.6240.4ee5.6211.4eec.53ef.4ee5.65b9.4fbf.7684.628a.78.5f.69.20.78.5f.6a.20.6620.5c04.5230.67d0.4e2a.65b0.51fd.6570.4e0a.ff0c.4e0d.6539.53d8.5176.20.4c.6f.73.73.20.7684.5355.8c03.6027.5373.53ef')

#### Q3: 决策树的 Loss 函数是什么？随机森林是什么？ 

回答:

In [4]:
#hint("""4e00.3001.71b5.7684.548c.6700.5c0f.ff0c.8981.9009.62e9.4e00.4e2a.6761.4ef6.8ba9.8fd9.6b21.5206.7c7b.7684.4e24.8fb9.7ed3.679c.7adf.53ef.80fd.7684.2018.7eaf.2019.2c.20.4f8b.5982.ff0c.6211.4eec.6709.5b.32.2c.20.31.2c.20.30.2c.20.30.2c.20.30.2c.20.30.2c.20.31.2c.20.31.2c.20.31.2c.20.31.2c.20.31.5d.a.5982.679c.6211.4eec.9009.62e9.4e00.4e2a.6761.4ef6.662f.27.662f.4e0d.662f.31.27.ff0c.6211.4eec.53ef.4ee5.5206.6210.5b.30.2c.20.30.2c.20.30.2c.20.30.2c.20.32.5d.2c.20.5b.31.2c.20.31.2c.20.31.2c.20.31.2c.20.31.5d.ff0c.20.4e5f.53ef.4ee5.6761.4ef6.662f.27.662f.4e0d.662f.32.27.2c.20.a.6211.4eec.5c31.53ef.4ee5.5206.6210.5b.32.5d.2c.20.5b.31.2c.20.30.2c.20.30.2c.20.30.2c.20.30.2c.20.31.2c.20.31.2c.20.31.2c.20.31.2c.20.31.5d.ff0c.20.663e.7136.524d.8005.66f4.7eaf.3002.20.5982.679c.540c.5b66.4eec.5fd8.4e86.71b5.7684.6982.5ff5.ff0c.5927.5bb6.8d76.7d27.518d.67e5.4e00.4e0b.ff0c.7136.540e.8ba1.7b97.4e00.4e0b.8fd9.4e24.4e2a.5206.7c7b.7ed3.679c.7684.71b5.662f.591a.5927.ff1b.a.4e8c.3001.968f.673a.68ee.6797.662f.7528.5f88.591a.5c0f.7684.51b3.7b56.6811.7528.6765.6295.7968.7684.96c6.6210.6a21.578b.ff08.45.6e.73.65.6d.62.6c.65.ff09.ff0c.6bcf.4e2a.5c0f.51b3.7b56.6811.4f7f.7528.4e00.90e8.5206.7684.20.66.65.61.74.75.72.65.20.8fdb.884c.8bad.7ec3""")

**插入**：我们在这里快速的过一下如何在 Jupyter 中写数学公式。 这个其实很简单，如果我们要输入一个公式，例如A1, 那么，我们在Jupyter中输入`$A_i$`, 然后 Enter， 是不是就变成了$A_i$? 其实两个`$$`之间的东西就是 Latex 的符号，`$..$`这个我们叫做inline模式，意思就是说你写出来的公式是和你的文字在一行里，如果你`$$..$$``，这个公式就会单独是一行。

我们现在再试一个, 输入`$$\frac{P_i}{\sum_{j \in \mathbf{V}}^NP_j}$$`, 输完之后 Enter， 你看到了什么？ 

$$\frac{P_i}{\sum_{j \in \mathbf{V}}^NP_j}$$

这个时候会有同学说，可是这些符号，我怎么记得住呢？ 我给大家提供了一个参考手册，大家有空就看看 https://github.com/Artificial-Intelligence-for-NLP/comment-setimental-classification/blob/master/Latex-Symbols.pdf，熟能生巧。 


#### Q4: 使用Latex 写出来决策树希望找到一个 feature，这个 feature 使得熵的和最少的公式。

回答：

##### Q5: 贝叶斯公式的原理是什么？ 我们现在用的贝叶斯分类器为什么是“朴素贝叶斯”， 它为什么朴素？

回答：

#### Q6: 神经网络的Loss函数的作用为何？

回答：loss函数表现了当前模型与真实答案的误差，在我们对loss函数求导后，我们可以使用梯度下降法+反向传播对当前模型中的参数进行优化。

In [5]:
hint("""795e.7ecf.7f51.7edc.91cc.9762.7684.4c.6f.73.73.51fd.6570.662f.7528.6765.8861.91cf.6a21.578b.7684.597d.574f.ff0c.4c.6f.73.73.51fd.6570.8d8a.5927.ff0c.9884.6d4b.4e0e.5b9e.9645.7684.8bef.5dee.8d8a.5927.ff0c.9884.6d4b.8d8a.4e0d.51c6.786e.3002.4e3a.4e86.8ba9.9884.6d4b.7ed3.679c.66f4.52a0.7cbe.51c6.ff0c.6211.4eec.8981.51cf.5c11.4c.6f.73.73.51fd.6570.ff0c.901a.8fc7.68af.5ea6.4e0b.964d.6cd5.ff0c.5229.7528.53cd.5411.4f20.64ad.4e0d.505c.8fed.4ee3.8c03.6574.795e.7ecf.7f51.7edc.4e2d.7684.53c2.6570.ff0c.627e.5230.4f7f.4c.6f.73.73.51fd.6570.6700.5c0f.7684.53c2.6570.ff0c.786e.5b9a.6a21.578b.3002""")

神经网络里面的Loss函数是用来衡量模型的好坏，Loss函数越大，预测与实际的误差越大，预测越不准确。为了让预测结果更加精准，我们要减少Loss函数，通过梯度下降法，利用反向传播不停迭代调整神经网络中的参数，找到使Loss函数最小的参数，确定模型。


#### Q7: 神经网络的激活函数(activation function)起什么作用？ 如果没有激活函数会怎么样？ 

回答：因为激活层的上层是线性变化，若是没有激活层，则整个模型就是一个线性模型，模型的拟合能力普通；但是有了激活层，使得模型可以进行非线性变化，让模型可以拟合任意的函数。

In [6]:
hint('6fc0.6d3b.51fd.6570.7528.6765.8fdb.884c.975e.7ebf.6027.53d8.5316.ff0c.4e0d.65ad.5f97.975e.7ebf.6027.53d8.5316.4f7f.5f97.6211.4eec.28.7406.8bba.4e0a.29.53ef.4ee5.62df.5408.4efb.610f.51fd.6570.ff0c.8fd9.4e5f.662f.4e3a.4ec0.4e48.795e.7ecf.7f51.7edc.80fd.60.5b66.4e60.60.7684.539f.56e0.3002.795e.7ecf.7f51.7edc.91cc.8fb9.7684.60.5b66.4e60.60.5176.5b9e.5c31.662f.51fd.6570.62df.5408.7684.610f.601d')

激活函数用来进行非线性变化，不断得非线性变化使得我们(理论上)可以拟合任意函数，这也是为什么神经网络能`学习`的原因。神经网络里边的`学习`其实就是函数拟合的意思


#### Q8: 神经网络的softmax如何理解， 其作用是什么？ 在`答案`中写出softmax的python表达；


回答：softmax跟在全连层的输出之后；它将全连层的输出先通过指数函数固定在（0，正无穷）的区间里，再将他们归一化到（0，1）的区间里以概率的形式表达全连层的输出，可解释性非常高。

百科：Softmax函数实际上是有限项离散概率分布的梯度对数归一化

In [7]:
import math
z = [2.0,3.0,10.0]
z_exp = [math.exp(i) for i in z]  
print(z_exp)  
sum_z_exp = sum(z_exp)
print(sum_z_exp) 
softmax = [round(i / sum_z_exp, 3) for i in z_exp]
print(softmax) 

[7.38905609893065, 20.085536923187668, 22026.465794806718]
22053.940387828836
[0.0, 0.001, 0.999]


#### Q9: 简述 normalized_1 和softmax函数的相同点和不同点， 说明softmax相比normalized_1该函数的优势所在

In [8]:
import numpy as np
y1=2 
y2=3
y3=10
output = np.array([y1, y2, y3])

normalized_1 = output / np.sum(output)
print(normalized_1)

[0.13333333 0.2        0.66666667]


回答：他们两者都可以将一组向量归一化到（0，1）的区间上，以一种概率的形式呈现结果。不同点，normalized_1简单的将数值累加，计算概率分布，只要数值的比例一样，则结果不变，不能体现出输出变化的情况

#### Q10: 写出crossentropy的函数表达式，说明该函数的作用和意义

回答：

..

..

----------------- 休息一下，接下来是关于 Word2Vec的 ------------- 

#### Q11: 说明word2vec要解决的问题背景， 以及word2vec的基本思路， 说明word2vec比起之前方法的优势；

回答：人们希望判断单词之间是否相似，所以通过向量来表征一个单词来解决这个问题；基本思路是
1. 输入是所有单词的one-hot向量以及分词完毕的文本，然后将one-hot向量和第一个权重矩阵（100*100）相乘，输出V1(1*100)。
2. V1和第二个权重矩阵(100*10000)相乘,输出（1*10000）的v2.
3. 将v2矩阵进行softmax，使得输出的向量每个值变成概率分布，获得概率向量V3.
4. 使用真实数据进行相似度测试，我们期望两个相似的词，它们的词向量也应该相似，乘积要大，根据这一标准我们重新计算并且得到loss，通过loss训练w1和w2的权重。

#### Q12: 说明word2vec的预测目标， predication target, 在答案中写出skip-gram和cbow的预测概率；

回答： 预测目标是词与词之间的相似度，skip-gram是用这一个词预测它周围的几个词，cbow是用周围的词来预测这一个词。

#### Q13: 请说明word2vec的两种常见优化方法，分别阐述其原理；

回答：Hierarchical Softmax + negative sampling
Hierarchical Softmax：本来从隐藏层到输出层要把所有的softmax计算完再找最大的那一个；由于我们把之前所有都要计算的从输出softmax层的概率计算变成了一颗二叉霍夫曼树，那么我们的softmax概率计算只需要沿着树形结构进行就可以了。缺点：若根节点就很复杂，那么整颗树都会变得很复杂。

负采样：用中心词w和他周围的词Content作为正例，使用负采样将其他w1和w1周围的词作为负例，进行回归计算。

#### Q14: 请说明word2vec中哈夫曼树的作用；

回答：霍夫曼树是二叉树的一种特殊形式，又称为最优二叉树，其主要作用在于数据压缩和编码长度的优化。
目的是想让常用的词，权重高的词拥有更短的编码，方便计算；而权重低的，不常用的词则远离根节点，编码较长。

#### Q15: 哈夫曼树如何构建？

In [9]:
hint('a.31.2e.20.68.74.74.70.73.3a.2f.2f.67.69.74.68.75.62.2e.63.6f.6d.2f.68.65.69.6e.65.6d.61.6e.2f.70.79.74.68.6f.6e.2d.64.61.74.61.2d.73.74.72.75.63.74.75.72.65.73.2f.62.6c.6f.62.2f.6d.61.73.74.65.72.2f.35.2e.25.32.30.48.65.61.70.2d.62.61.73.65.64.25.32.30.53.74.72.75.63.74.75.72.65.73.2f.68.75.66.66.6d.61.6e.2e.70.79.a.32.2e.20.68.74.74.70.73.3a.2f.2f.67.69.74.68.75.62.2e.63.6f.6d.2f.52.61.52.65.2d.54.65.63.68.6e.6f.6c.6f.67.69.65.73.2f.67.65.6e.73.69.6d.2f.62.6c.6f.62.2f.33.64.35.61.32.31.63.31.63.38.31.32.38.63.62.38.64.64.34.66.36.65.35.31.65.39.65.66.33.64.63.35.61.66.30.30.30.38.37.31.2f.67.65.6e.73.69.6d.2f.6d.6f.64.65.6c.73.2f.64.65.70.72.65.63.61.74.65.64.2f.77.6f.72.64.32.76.65.63.2e.70.79.23.4c.36.37.30.22.a.33.2e.20.68.74.74.70.73.3a.2f.2f.77.77.77.2e.77.69.6b.69.77.61.6e.64.2e.63.6f.6d.2f.65.6e.2f.48.75.66.66.6d.61.6e.5f.63.6f.64.69.6e.67.a')


1. https://github.com/heineman/python-data-structures/blob/master/5.%20Heap-based%20Structures/huffman.py
2. https://github.com/RaRe-Technologies/gensim/blob/3d5a21c1c8128cb8dd4f6e51e9ef3dc5af000871/gensim/models/deprecated/word2vec.py#L670"
3. https://www.wikiwand.com/en/Huffman_coding



#### Q16: 在gensim中如何实现词向量？ 请将gensim中实现词向量的代码置于答案中

回答：
from gensim.models import Word2Vec
sentences = [["cat", "say", "meow"], ["dog", "say", "woof"]]
model = Word2Vec(sentences, min_count=1)
model.wv.vocab
model.most_similar('cat')

#### Q17: 请说出除了 skip-gram和cbow的其他4中词向量方法的名字， 并且选取其中两个叙述其基本原理。

回答：Onehot: 用和词汇表相同的矩阵大小来通过0-1编码表示一个词。
EMLo：在每一层RNN都进行一个提取，最后将结果线性组合后输出。

In [10]:
hint('a.4f.6e.65.68.6f.74.2c.20.47.6c.6f.76.65.2c.43.6f.76.65.2c.45.4d.4c.6f.a')


Onehot, Glove,Cove,EMLo



-------------------- 休息一下，接下来是关于 Keras 和 Tensorflow 使用的 -------------- 

大家先熟悉一下什么是MNIST数据集： 

> http://yann.lecun.com/exdb/mnist/

>https://en.wikipedia.org/wiki/MNIST_database

#### Q18: 参考keras参考手册，构建一个机器学习模型，该模型能够完成使用DNN(deep neural networks) 实现MNIST数据集的分类；

关键代码: 做完了

#### Q19:参考tensorflow的参考手册，构建一个机器学习模型，该模型能够完成使用DNN(deep neural networks)实现MNIST数据集的分类；

关键代码：做完了

In [18]:
hint('68.69.6e.74.73.3a.74.65.6e.73.6f.72.66.6c.6f.77.5b9e.73b0.4d.4e.49.53.54.20.68.74.74.70.73.3a.2f.2f.67.69.74.68.75.62.2e.63.6f.6d.2f.74.65.6e.73.6f.72.66.6c.6f.77.2f.74.65.6e.73.6f.72.66.6c.6f.77.2f.62.6c.6f.62.2f.6d.61.73.74.65.72.2f.74.65.6e.73.6f.72.66.6c.6f.77.2f.65.78.61.6d.70.6c.65.73.2f.75.64.61.63.69.74.79.2f.32.5f.66.75.6c.6c.79.63.6f.6e.6e.65.63.74.65.64.2e.69.70.79.6e.62')

hints:tensorflow实现MNIST https://github.com/tensorflow/tensorflow/blob/master/tensorflow/examples/udacity/2_fullyconnected.ipynb


#### Q20: 参考keras和tensorflow对同一问题的实现，说明keras和tensorflow的异同；

回答：keras是tensorflow的高级集成版本，直接向用户提供接口；tensorflow作为深度学习的API大头已经将很多操作集成起来，提供了统一的API接口，这些接口更多的是一些细致的点。

#### Q21: tensorflow 使用 Graph 计算机制的优缺点是什么？ 

回答：图描述了一组需要依次序完成的计算单元以及表示这些计算单元之间相互依赖的关系；图的信息流非常清晰，让网络框架一目了然。缺点：要先编译这张图才能计算，python运行速度一般？

#### Q22: Q18， Q19 的tensorflow 或 keras 模型的训练时准确率和测试集准确率分别是多少？

回答：99%

#### Q23: 训练时准确率大于测试集准确率的现象叫什么名字，在神经网络中如何解决该问题？(至少提出5个解决方法)

回答：过拟合
1. Dropout
2. L0-2正则
3. 增加训练集
4. 清洗数据
5. 减小模型复杂度
6. 减少训练时间

#### Q24: 请使用自己的语言简述通过正则化 (regularization)减小过拟合的原理；

回答： 通过样本数量以及超参数数量这种超参数给模型规定一定的变化范围，让他们不会无限的去拟合训练数据。给模型加上限制。
1. L1:
2. L2:

#### Q25: 在tensorflow官方实例中给出的fully connected 神经网络的分类模型中，数据进行了哪些预处理，这些预处理的原因是什么？

回答：

--------------------- 休息一下，接下来是关于 RNN 和 CNN 的 ---------- 

#### Q26: 简述CNN的原理

回答：

#### Q27: CNN的 Spatial Invariant是什么意思？ 是如何做到的？ 

回答：

#### Q28: CNN增加了很多层数，这些层数使用 filter 进行计算。 按说需要拟合的参数变得很多，请问 CNN 是如何解决这个问题的，如何加快速度的？ 

回答：

In [15]:
#hint('a.6d.61.69.6e.20.70.6f.69.6e.74.73.3a.20.50.6f.6f.6c.69.6e.67.2c.20.50.61.72.61.6d.65.74.65.72.20.53.68.61.72.69.6e.67.a')

#### Q29: CNN中的 Batch Normalization有什么意义？

回答:

#### Q30: CNN中的 Pooling 起到什么作用？

回答：

#### Q31: CNN中的 Fully Connect起到什么作业？

回答：

#### Q32: 深度网络中的权值初始化有什么讲究？

回答：

#### Reading: 参照 Keras 和 Tensorflow 的示例，手敲使用 keras, tensorflow + CNN 实现MNIST分类的问题：

+ https://github.com/keras-team/keras/blob/master/examples/mnist_cnn.py
+ https://github.com/tensorflow/tensorflow/blob/master/tensorflow/examples/udacity/4_convolutions.ipynb

把代码手敲一遍

#### Q33: 简述RNN解决的问题所具有的特点；

回答：

#### Q34: 写出RNN实现时间或者序列相关的数学实现(见课程slides)

回答：

#### Q35: 简述RNN的两种重要变体的提出原因和基本原理？

回答: 

#### Q36:  Attentional RNN 以及 Stacked RNN 和 Bi-RNN 分别是什么，其做了什么改动？

回答：

#### Reading 和 CNN 类似，请在 Keras,Tensorflow中查找如何实现 RNN 模型

+ https://github.com/Artificial-Intelligence-for-NLP/References/blob/master/AI%20%26%20Machine%20Learning/Hands.On.TensorFlow.pdf

## 第二部分： 项目解决过程

代码主要在 Pycharm 里边写，jupyter 里边写一个关键步骤就行

#### Q37: 要实现文本分类或情感分类，文本信息需要进行哪些初始化操作？自己手工实现，keras提供的API，tenorflow提供的API，分别是哪些？请提供关键代码置于下边`回答`中

回答：

In [17]:
#hint('id_to_word, word_to_id, padding, batched')

ValueError: invalid literal for int() with base 16: 'id_to_word, word_to_id, padding, batched'

#### Q38 在没有预训练的词向量时候， keras 如何实现embedding操作，即如何依据一个单词的序列获得其向量表示？

回答： 先将单词进行id表征，然后补0使得输入的长度一致，然后使用embedding层进行词嵌入操作。

#### Q 39: 在**有**预先训练的词向量时候，keras和tensorflow又如何实现embeding操作

回答：

#### Q40：基于上文进行的数据预处理，使用keras和tensorflow如何构建神经网络模型？请提供关键代码

### 好的 现在开始切入正题 --

其实，我们解决实际问题的时候，很少自己从头到尾写一个神经网络模型，我们往往是找一个效果比较好的类似问题的模型，然后在这个问题上改造。 或者我们在去一个公司的时候，接手的工作也往往是改动以前的模型，所以我们解决这个语义分类问题我们也首先是找一个类似的问题，然后参考一个模型进行修改，变成能够解决我们这个问题的模型。

我们以上所以的理论知识，都是用来支持我们做修改，能够看懂别人为何要这样写，然后自己要改哪里。 

kaggle上的“恶意评价识别”这个项目和我们的这个项目是类似的, 大家请首先在这个的 Kernel 里边找到一个公开代码的示例，然后选择一个自己能够看懂且效果较好的模型进行改造。

+ https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge
+ https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge/kernels


Kaggle这个问题和我们的问题类似，但是并不是完全一样， 其中最不一样的其实是我们的期望的结果这里, Kaggle 这里的输出是5个类别，然后类别的是0~1直接的数字来预测是否是这个类别，然后我们的客户评价问题中，打标对20个分类的-2, -1, 0, 1四个标记. 一种最简单的方法，是把这个20分类问题变成80分类问题，然后每个分类的输出是0或者1. 

#### Q41. 依据Kernel 中选择方法，对数据和代码进行改造，使其符合选择该问题。

回答：选择了一个Bi-LSTM+全连层的网络。改变了输入输出的维度后，初步进行训练。

#### Q42. 你现在的模型的准确率是多少？ 如何知道你的模型是不是真的学习了 而不是随机的进行猜测？

回答：0.41, 非常低，我认为有数据集的问题，同时也有模型的问题，处于欠拟合的状态。

#### Q43. 你的模型现在准确度不高的原因，你猜测主要是什么？ 

回答：模型不够复杂，无法很好的提取特征然后判断20个分类。

#### Q43. 如前文所述，这个问题很难，其实现在也没有什么万灵药方法。 所以需要同学们多想想如何有效， 可以给大家参考的优化方式有， 修改vocabulary size, embedding size,去掉停用词，重新组合词组等。 并且结合使用LSTM， GRU， Bi-RNN， Stacked， Attentional, regularization, 等各种方法组合进行模型的优化， 至少进行10次优化，每次优化请按照以下步骤填写：

回答：

回答：

---这是一个实例----

第1次优化：

1. 存在的问题： loss下降太慢；
2. 准备进行的优化：减小模型的神经单元数量；
3. 期待的结果：loss下降加快；
4. 实际结果：loss下降的确加快(或者并没有加快)
5. 原因分析：模型神经元数量减小，收敛需要的次数减少，loss下降加快


---你的实验优化结构记录在此---

**第1次优化**：

1. 存在的问题： 准确度41%
2. 准备进行的优化：将分类变成80个二元分类。
3. 期待的结果：准度提高
4. 实际结果：准确度到了70%
5. 原因分析：分类器无法处理多值分类？或者可能是loss计算时有问题，对于1，2，3，4无法很好的得出loss。

**第2次优化**：

1. 存在的问题： 准确度70%
2. 准备进行的优化：加大输入训练数据到40000个，并且打乱随机顺序
3. 期待的结果：准度提高
4. 实际结果：acc 83%,f1 66%
5. 原因分析：打乱数据以及增加样本让模型学习到了更多的知识，同时对每一个类别的数据都有学习。

**第3次优化**：

1. 存在的问题： acc 83%,f1 66%
2. 准备进行的优化：增加一层或者二层Bi-LSTM层。
3. 期待的结果：准度提高
4. 实际结果：acc 0.84, f1 67
5. 原因分析：没有什么提高了

**第4次优化**：

1. 存在的问题： acc 0.84, f1 67
2. 准备进行的优化：增大Embedding的矩阵大小，从256提高到400，把所有的LSTM换成了GRU
3. 期待的结果：准度提高
4. 实际结果：acc 0.843, f1 68
5. 原因分析：并不影响？现有模型已经不能从中再提取到更多的信息

**第5次优化**：

1. 存在的问题： 
2. 准备进行的优化：
3. 期待的结果：
4. 实际结果：
5. 原因分析：

**第6次优化**：

1. 存在的问题： 
2. 准备进行的优化：
3. 期待的结果：
4. 实际结果：
5. 原因分析：

**第7次优化**：

1. 存在的问题： 
2. 准备进行的优化：
3. 期待的结果：
4. 实际结果：
5. 原因分析：

**第9次优化**：

1. 存在的问题： 
2. 准备进行的优化：
3. 期待的结果：
4. 实际结果：
5. 原因分析：

**第10次优化**：

1. 存在的问题： 
2. 准备进行的优化：
3. 期待的结果：
4. 实际结果：
5. 原因分析：


## 最后一步： 使用Flask、Bottle、Bootstrap变成一个网络应用并且部署在服务器上，这样别人就可以直接通过网址访问你的应用啦。

最后一步，我们使用Bottle，Bootstrap,Flask等工具进行可视化现实，做出网页能够访问的形式，就像我们的第一个项目一样 😁.

## 本次项目的总结

请写项目的总结报告，描述此次项目的主要过程，其中遇到的问题，以及如何解决这些问题的，以及有什么经验和收获。

恭喜你，你完成了一个**十分**复杂的问题， 能完成这个问题，求是求是，你的能力其实已经达到了国内绝大多数公司的要求，你缺的只是熟练程度。 多多在 Kaggle， 阿里天池里边找一些自己感兴趣的问题，多练习练习。 熟能生巧。